In [2]:
# -*- coding: cp1250 -*-
import serial
import logging
import sys
import os
 
 
def to_hex_word(a):
    '''encodes a decimal number hexadecimally on two bytes'''
    return a.to_bytes(2,byteorder=sys.byteorder)
 
class SerialPort(object):
    def __init__(self, port_name):
        import serial
        try:
            self.port = serial.Serial(
                port=port_name,
                baudrate=9600,
                bytesize=serial.EIGHTBITS,
                parity=serial.PARITY_NONE,
                stopbits=serial.STOPBITS_ONE,
                xonxoff=False
                )
        except serial.SerialException as e:
            print("Nieprawidłowa nazwa portu lub port zajęty.")
            raise e
        self.close()
 
    def open(self):
        self.port.open()
 
    def close(self):
        self.port.close()
 
    def send(self, value):
        self.port.write(value)
 
    def blinkSSVEP(self,d, p1, p2):
        '''
        d = list of frequencies;
        p1:p2 = ratio LED_on_time/LED_off_time
        if you want i-th LED to be OFF all the time send  d[i] = 0
        if you want i-th LED to be ON all the time send  d[i] = -1
        in these two cases p1 and p2 do not matter
        '''
        clock  = 62500
        factor = float(p1) / float(p1 + p2)
 
        str = (3).to_bytes(1,byteorder=sys.byteorder) # 'SSVEP_RUN'
 
        for i in range(len(d)):
            # i-th LED OFF
            if d[i] == 0:                       
                str += to_hex_word(0) + to_hex_word(255) 
            # i-th LED ON
            elif d[i] == -1:
                str += to_hex_word(255) + to_hex_word(0)
                #str = 'S'
                # i-th LED blinks d[i] times per second
                # p1:p2 = on_time:off_time in one blink
            else:
                period = clock/d[i]
                bright = int((clock/d[i]) * factor)
                dark = int(period - bright)
                str += to_hex_word(bright) + to_hex_word(dark)
 
        self.send(str)
 
    def blinkP300(self,d):
        clock  = 62500
        str = (4).to_bytes(1,byteorder=sys.byteorder) # 'P300_RUN'
 
        for i in range(len(d)):
            period = int(clock*d[i]/1000.0)
            str += to_hex_word(period)
 
        self.send(str)

In [3]:
sp = SerialPort('/dev/ttyUSB0')
sp.open()
sp.blinkSSVEP([10, 0],1,1)

In [23]:
import numpy as np
import time
import datetime

In [7]:
freqs = np.array([4, 7, 10, 13, 16, 20, 25, 30, 35, 40])

In [28]:
timestamp = datetime.datetime.now()
text_file = open(f"kolejnosc_{timestamp}.csv", "w")
text_file.write("Index,Częstość,Spanko\n")
print("Index,Częstość,Spanko")
for i in range(0, 15):
    temp_freqs = freqs.copy()
    np.random.shuffle(temp_freqs)
    for j in range(0, len(freqs)):
        freq = temp_freqs[-1]
        temp_freqs = temp_freqs[:-1]
        sp.blinkSSVEP([freq, 0],1,1)
        time.sleep(5)
        sp.blinkP300([0,0])
        sleep_time = np.random.randint(3,8)
        text_file.write(f"{i*j+j},{freq},{sleep_time}\n")
        print(f"{i*j+j},{freq},{sleep_time}")
        time.sleep(sleep_time)
text_file.close()
        

Index,Częstość,Spanko
0,10,7
1,16,3
2,35,3
3,40,7
4,20,6
5,13,3
6,4,4
7,7,7
8,25,5
9,30,4
0,25,5
2,4,5
4,35,7
6,13,7
8,20,4
10,16,7
12,10,3
14,7,7
16,30,5
18,40,5
0,20,7
3,25,5
6,7,3
9,4,3
12,40,4
15,10,4
18,16,7
21,13,7
24,35,3
27,30,4
0,25,4
4,16,7
8,30,5
12,13,4
16,20,4
20,35,7
24,40,6
28,7,5
32,4,3
36,10,6
0,7,3
5,13,4
10,16,7
15,4,5
20,30,4
25,10,7
30,20,4
35,35,5
40,25,4
45,40,7
0,7,4
6,25,5
12,10,7
18,35,6
24,40,6
30,4,6
36,13,3
42,30,7
48,16,3
54,20,5
0,4,6
7,30,3
14,7,3
21,16,6
28,25,6
35,20,6
42,10,5
49,35,4
56,13,4
63,40,5
0,20,3
8,25,6
16,7,7
24,13,5
32,10,3
40,4,5
48,40,7
56,16,6
64,30,7
72,35,4
0,4,5
9,10,7
18,40,6
27,16,5
36,20,7
45,35,3
54,30,5
63,25,4
72,13,6
81,7,4
0,4,4
10,40,5
20,7,7
30,20,4
40,10,7
50,16,5
60,25,5
70,13,7
80,30,7
90,35,6
0,35,4
11,16,4
22,7,3
33,4,7
44,30,3
55,20,3
66,25,3
77,40,6
88,10,6
99,13,7
0,40,6
12,35,6
24,13,3
36,10,4
48,7,5
60,4,5
72,20,3
84,25,3
96,30,3
108,16,4
0,7,6
13,4,7
26,16,6
39,40,3
52,35,6
65,30,5
78,13,7
91,25,6
104,10,3
117,20

In [27]:
sp.blinkP300([0, 0])

In [14]:
_8

NameError: name '_8' is not defined

In [16]:
print(Out[2])

KeyError: 2

In [33]:
%hist -g -o -f "historia_3.txt"